In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pandas import DataFrame
from nemosis import dynamic_data_compiler
import glob
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

participant_duid_df = pd.read_csv('../data/Participants.csv')

# USE PYTHON API FOR DOWNLOADING TABLES
raw_data_cache = '../data/nemosis_cache'
dispatch_load = dynamic_data_compiler(start_time='2023/10/01 00:00:00', end_time='2023/11/01 00:05:00', table_name='DISPATCHLOAD', raw_data_location=raw_data_cache)
print("Dispatch Load:")
print(dispatch_load)

nam_parquet_dir = '../data/nam_parquet'

# Get a list of all parquet files in the directory
files = glob.glob('../data/nam_parquet/*.parquet')

# Read each file and concatenate them
df_price_parquet = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True)

start_date = datetime.strptime("01/10/2023 00:00", "%d/%m/%Y %H:%M")
end_date = datetime.strptime("01/11/2023 00:05", "%d/%m/%Y %H:%M")

df_price_parquet = df_price_parquet[(df_price_parquet['SETTLEMENTDATE'] > start_date) & (df_price_parquet['SETTLEMENTDATE'] <= end_date)]
dispatch_price = df_price_parquet

print('Composite Dataframe')
composite_df = pd.merge(participant_duid_df, dispatch_load, on='DUID', how='inner')
print(composite_df)

# Merge the DataFrames on 'Region' and 'SETTLEMENT'
final_df = pd.merge(composite_df, dispatch_price, 
                    left_on=['Region', 'SETTLEMENTDATE'], 
                    right_on=['REGIONID', 'SETTLEMENTDATE'], 
                    how='left')

# Drop the redundant 'RegionID' column from the merge
final_df = final_df.drop(columns=['REGIONID', 'INTERVENTION_y', 'INTERVENTION_x', 'LASTCHANGED', 'DOWNLOAD_TIME', 'PRICE_STATUS', 'SEMIDISPATCHCAP', 'RAISEREGENABLEMENTMAX', 'RAISEREGENABLEMENTMIN', 'LOWERREGENABLEMENTMAX', 'LOWERREGENABLEMENTMIN', 'SEMIDISPATCHCAP'])

one_sec_columns = ['RAISE1SEC', 'LOWER1SEC', 'RAISE1SECRRP', 'LOWER1SECRRP']
for col in one_sec_columns:
    if col in final_df.columns and final_df[col].isnull().any():
        final_df[col] = final_df[col].fillna(0)

composite_before_revenue_df = final_df.sort_values(by='SETTLEMENTDATE').reset_index(drop=True)

# Create csv file of final_df
print("Composite Dataframe Before Revenue: ")
print(composite_before_revenue_df)

In [ ]:
from batteries import *

battery_revenues = {}

mappings = {
    'Hornsdale Power Reserve': HornsdaleBattery,
    'Victorian Big-Battery': VictorianBigBattery,
    'Dalrymple North BESS': DalrympleNorthBESS,
    'Lake Bonney BESS1': LakeBonneyBESS,
    'Wandoan Battery Energy Storage System (BESS)': WandoanBESS,
    'Ballarat Battery Energy Storage System': BallaratBESS,
    'Wallgrove BESS 1': WallgroveBESS,
    'Gannawarra Energy Storage System': GannawarraBESS,
    'Torrens Island BESS': TorrensIslandBESS,
    'Hazelwood Battery Energy Storage System': HazelWoodBESS,
    'Bouldercombe Battery Project': BouldercombeBESS,
    'Riverina Energy Storage System 1': RiverinaEnergyStorageSystem1,
    'Riverena Energy Storage System 2': RiverenaEnergyStorageSystem2,
    'Bulgana Green Power Hub': BulganaGreenPowerHub,
    'Queanbeyan BESS': QueanbeyanBESS,
    'Darlington Point Energy Storage System': DarlingtonPointEnergyStorageSystem,
    'Chinchilla BESS, Units 1-80': ChinchillaBESS,
    'Broken Hill Battery Energy Storage System': BrokenHillBESS,
    'Capital Battery': CapitalBattery,
    'Tailem Bend 2 Hybrid Renewable Power Station': TailemBend2,
    'Western Downs Battery Energy Storage System': WesternDownsBESS,
    'Adelaide Desalination Plant': AdelaideDesalinationPlant,
    'Blyth Battery Energy Storage System': BlythBESS,
    'Christies Beach Wastewater Treatment Plant': ChristiesBeachWastewaterTreatmentPlant,
    'Happy Valley Water Treatment Plant': HappyValleyWaterTreatmentPlant,
    'Kennedy Energy Park Battery': KennedyEnergyParkBattery,
    'Phillip Island BESS': PhillipIslandBESS,
    'Rangebank BESS': RangebankBESS
}

for key, value in mappings.items():
    if key in composite_before_revenue_df['Station Name'].values:
        instance = value(composite_before_revenue_df)
        revenue_df = instance.add_revenue(instance.battery)
        revenue_df = revenue_df.dropna()
        battery_revenues[value.__name__ + '_df'] = revenue_df

In [ ]:
from visualizers import *

data = battery_revenues['LakeBonneyBESS_df']

# Example: Plot interactive bar graph by day
plot_energy_revenue(data, 'Day')
plot_regulation_revenue(data, 'Day')
plot_contingency_revenue(data, 'Day')